In [202]:
# Loading modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [203]:
# Loading data from Excel
df = pd.read_excel('data/LV_Datenbank.xlsx', sheet_name='Daten Trägersysteme 2021')

In [204]:
# Creating a dataframe with all fuel types
df_fuel = pd.DataFrame(pd.unique(df[['stage1_fuel', 'stage2_fuel', 'stage3_fuel', 'stage4_fuel', 'stage5_fuel']].values.ravel('K')), columns=['fuel'])

# Dropping NaN Value
df_fuel = df_fuel.dropna()

# Setting Fuel as index
df_fuel = df_fuel.set_index('fuel')

Empty DataFrame
Columns: []
Index: [RP-1/LOX, Al/AP/HTPB, HTPB, N2O4/UDMH, LH2/LOX, Jet A/LOX, TH-H8299/Al, Hydrazine, MMH/MON]


In [205]:
# Adding the stoichiometric combustion products per kg fuel

# ['RP-1/LOX', 'Al/AP/HTPB', 'HTPB', 'N2O4/UDMH', 'LH2/LOX', 'Jet A/LOX', 'TH-H8299/Al', 'Hydrazine', 'MMH/MON']
df_fuel['CO2'] = np.array([3.15, 0.385690653, 3.214088774, 1.464583369, 0, 3.15, 0, 0, 0.955238504])
df_fuel['H2O'] = np.array([1.26, 0.277903767, 0.993343379, 1.199053817, 8.936682739, 1.26, 0, 1.124368235, 1.17308007])
df_fuel['N2'] = np.array([0, 0.0140067, 0, 1.398378524, 0, 0, 0, 1.311277585, 2.736174062])
df_fuel['O2'] = np.array([0, 0.23490668, 0, 0, 0, 0, 0, 0, 0])
df_fuel['HCl'] = np.array([0, 0.214130989, 0, 0, 0, 0, 0, 0, 0])
df_fuel['Al2O3'] = np.array([0, 0.358998351, 0, 0, 0, 0, 0, 0, 0])

print(df_fuel)

                  CO2       H2O        N2        O2       HCl     Al2O3
fuel                                                                   
RP-1/LOX     3.150000  1.260000  0.000000  0.000000  0.000000  0.000000
Al/AP/HTPB   0.385691  0.277904  0.014007  0.234907  0.214131  0.358998
HTPB         3.214089  0.993343  0.000000  0.000000  0.000000  0.000000
N2O4/UDMH    1.464583  1.199054  1.398379  0.000000  0.000000  0.000000
LH2/LOX      0.000000  8.936683  0.000000  0.000000  0.000000  0.000000
Jet A/LOX    3.150000  1.260000  0.000000  0.000000  0.000000  0.000000
TH-H8299/Al  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
Hydrazine    0.000000  1.124368  1.311278  0.000000  0.000000  0.000000
MMH/MON      0.955239  1.173080  2.736174  0.000000  0.000000  0.000000


In [206]:
# Replacing NaN with 0 within the original dataframe for further calculations
df = df.fillna(0)

In [228]:
# Creating a dataframe based on the factors of "df_fuel" and the used fuel type from "df"
# Multipling it by the stage mass for every product and every stage
for stage in np.arange(1, 6):
    for product in df_fuel.columns:
        df_temp = []
        for fuel_type in df['stage{0}_fuel'.format(stage)]:
            df_temp.append(df_fuel[product][fuel_type])
        df['stage{0}_{1}'.format(stage, product)] = df['stage{0}_mass_fuel'.format(stage)]*df_temp

df


,LV_type,No_stage,stage1_fuel,stage1_mass_fuel,stage1_mass_ox,stage1_dry_mass,stage1_burntime,stage2_fuel,stage2_mass_fuel,stage2_mass_ox,...,stage4_N2,stage4_O2,stage4_HCl,stage4_Al2O3,stage5_CO2,stage5_H2O,stage5_N2,stage5_O2,stage5_HCl,stage5_Al2O3
0,Angara A5/Persei,3.0,RP-1/LOX,147360.0,383040.0,39200.0,214.0,RP-1/LOX,36840.0,95760.0,...,7691.081882,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,Antares 230+,2.0,RP-1/LOX,64740.0,177260.0,20600.0,215.0,TH-H8299/Al,12837.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
2,Ariane 5ECA+,3.0,Al/AP/HTPB,240000.0,0.0,66000.0,140.0,LH2/LOX,33515.0,139925.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
3,Astra Rocket 3.3,2.0,RP-1/LOX,0.0,0.0,0.0,130.0,RP-1/LOX,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
4,Atlas V 401,2.0,RP-1/LOX,81168.0,202921.0,22825.0,253.0,LH2/LOX,5207.0,15623.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
5,Atlas V 421,3.0,HTPB,88400.0,0.0,10200.0,94.0,RP-1/LOX,81168.0,202921.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
6,Atlas V 551,3.0,HTPB,221000.0,0.0,25500.0,94.0,RP-1/LOX,81168.0,202921.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
7,Chang Zheng 2C,2.0,N2O4/UDMH,39444.0,102556.0,9000.0,132.0,N2O4/UDMH,24100.0,11500.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
8,Chang Zheng 2C/YZ-1S,3.0,N2O4/UDMH,39444.0,102556.0,9000.0,132.0,N2O4/UDMH,14850.0,38750.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
9,Chang Zheng 2D,2.0,N2O4/UDMH,124100.0,59100.0,9500.0,170.0,N2O4/UDMH,24100.0,11500.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0


In [ ]:
for product in df_fuel.columns:
    df_temp = []
    for fuel in df["stage1_fuel"]:
        df_temp.append(df_fuel[product][fuel])
    df['stage1_{0}'.format(product)] = df['stage1_mass_fuel']*df_temp
